# Run Ludo with Pygame in Google Colab

Welcome! This notebook allows you to run the graphical (Pygame) version of the Ludo game directly in your browser through Google Colab.

### How It Works
Because Colab is a cloud environment without a screen, we need to create a *virtual* display. This notebook will:
1.  **Install Dependencies**: Set up all necessary system and Python packages (like Pygame).
2.  **Run a Virtual Display (VNC)**: Start a virtual desktop that you can connect to from your browser.

---

## Step 1: Install Dependencies

This cell installs all the required packages. It might take a minute or two to complete.

In [ ]:
# Install Linux packages for virtual display and window management
!apt-get update > /dev/null 2>&1
!apt-get install -y xvfb x11vnc python3-opengl fluxbox > /dev/null 2>&1

# Install Python packages
!pip install -r requirements.txt > /dev/null 2>&1
!pip install pyvirtualdisplay pyngrok > /dev/null 2>&1

## Step 2: Launch the Game!

This final cell starts the virtual display and the Ludo game. It uses `ngrok` to create a secure link for you to access the game.

**Instructions:**
1.  **Get an ngrok Authtoken**: Go to [https://dashboard.ngrok.com/get-started/your-authtoken](https://dashboard.ngrok.com/get-started/your-authtoken) and copy your token. You only need to do this once.
2.  **Run the cell below**: Paste your authtoken into the input box that appears.
3.  **Click the Link**: After a moment, a public `ngrok.io` link will be printed. Click it to open the virtual desktop.
4.  **Enter Password**: The VNC password is `ludo`.
5.  **Play**: The Ludo game window should appear automatically. If not, you can start it from the terminal inside the VNC desktop (`python -m apps.gui.pygame_app`).

In [ ]:
#@title 🚀 Launch the Game!
#@markdown ### Enter your ngrok Authtoken
#@markdown You can get a free token from [https://dashboard.ngrok.com/get-started/your-authtoken](https://dashboard.ngrok.com/get-started/your-authtoken)
NGROK_AUTH_TOKEN = "" #@param {type:"string"}

import os
import subprocess
import time
from pyvirtualdisplay import Display
from pyngrok import ngrok

if not NGROK_AUTH_TOKEN:
  print("Error: Please enter your ngrok authtoken.")
else:
  # --- Setup ---
  os.environ['NGROK_AUTH_TOKEN'] = NGROK_AUTH_TOKEN
  ngrok.set_auth_token(NGROK_AUTH_TOKEN)

  # Start virtual display
  display = Display(visible=0, size=(1024, 768))
  display.start()

  # VNC password setup
  password = 'ludo'
  vnc_dir = os.path.expanduser('~/.vnc')
  os.makedirs(vnc_dir, exist_ok=True)
  subprocess.run(f"x11vnc -storepasswd {password} {os.path.join(vnc_dir, 'passwd')}", shell=True, check=True, capture_output=True)

  # Launch VNC server
  vnc_command = f'x11vnc -display {display.display} -forever -rfbauth {os.path.join(vnc_dir, "passwd")} -create -rfbport 5900 -bg'
  subprocess.Popen(vnc_command, shell=True)
  time.sleep(2)

  # Launch noVNC client
  novnc_command = '/usr/share/novnc/utils/launch.sh --vnc localhost:5900 --listen 6080'
  subprocess.Popen(novnc_command, shell=True)
  time.sleep(2)

  # --- Create ngrok tunnel ---
  try:
    ngrok.kill()
    public_url = ngrok.connect(6080, "http")
    print(f"✅ VNC connection is live!")
    print(f"👉 Click this link to open the virtual desktop: {public_url}")
    print(f"🔑 VNC Password: {password}")

    # --- Launch the Game ---
    game_command = 'python -m apps.gui.pygame_app'
    subprocess.Popen(
        game_command,
        shell=True,
        env=dict(os.environ, DISPLAY=display.display)
    )
    print("\n🚀 The Ludo game should now be running inside the virtual desktop.")
    print("\nℹ️ If the game window doesn't appear, you can open a terminal in the VNC desktop (right-click -> Terminal) and run the command `python -m apps.gui.pygame_app`")
    print("\n🛑 To stop the game and the server, interrupt or stop this Colab cell.")

  except Exception as e:
    print(f"An error occurred: {e}")
    print("This might be due to an invalid ngrok authtoken or a network issue.")
    print("Please check your token and try running the cell again.")